In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

In [6]:
train1 = pd.read_csv('./uv_train_cloud.csv')
train1['rainfall'].isnull().sum()
train1 = train1.drop(['rainfall'], axis=1)

group1_stn = list(train1['stn'].values)
stn1 = []
for v in group1_stn:
    if v not in stn1:
        stn1.append(v)
print(stn1)

stn1 = [105, 108, 112, 115, 131, 133, 
        138, 143, 146, 152, 156, 159, 165]
stn2 = [13, 132]

uv_train1 = train1[train1['stn'].isin(stn1)]
uv_train2 = train1[train1['stn'].isin(stn2)]

print(train1.columns)

[13, 105, 108, 112, 115, 131, 132, 133, 138, 143, 146, 152, 156, 159, 165]
Index(['yyyymmdd', 'hhnn', 'stn', 'lon', 'lat', 'uv', 'band1', 'band2',
       'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9', 'band10',
       'band11', 'band12', 'band13', 'band14', 'band15', 'band16', 'solarza',
       'sateza', 'esr', 'height', 'landtype', 'total_cloud', 'ml_cloud'],
      dtype='object')


In [7]:
###stn2 train set 먼저
train1 = train1.drop(['total_cloud', 'ml_cloud'], axis=1)


uv_train1_val = train1[(train1['yyyymmdd'] >= 20210501) & (train1['yyyymmdd'] < 20210801)]
idx1 = uv_train1_val.index
all_group1 = train1.drop(train1.index[idx1])


# cat_features = ['lat','height','landtype']

# for i in enumerate (cat_features) :
#     ca = i[1]
#     all_group1[ca] = all_group1[ca].astype('category')
#     uv_train1_val[ca] = uv_train1_val[ca].astype('category')



X = all_group1[['hhnn', 'lon', 'lat', 'band1', 'band2',
       'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9', 'band10',
       'band11', 'band12', 'band13', 'band14', 'band15', 'band16', 'solarza',
       'sateza', 'esr', 'height', 'landtype']]
Y = all_group1['uv']

x = uv_train1_val[['hhnn', 'lon', 'lat', 'band1', 'band2',
       'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9', 'band10',
       'band11', 'band12', 'band13', 'band14', 'band15', 'band16', 'solarza',
       'sateza', 'esr', 'height', 'landtype']]
y = uv_train1_val['uv']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123)

X_train = pd.concat([X,X_train])
y_train = pd.concat([Y,y_train])


# cat_features = ['lat','height','landtype']

# for i in enumerate (cat_features) :
#     ca = i[1]
#     X_train[ca] = X_train[ca].astype('category')
#     y_train[ca] = y_train[ca].astype('category')


In [54]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58245 entries, 1094810 to 1171572
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   hhnn      58245 non-null  int64   
 1   lon       58245 non-null  float64 
 2   lat       58245 non-null  category
 3   band1     58245 non-null  float64 
 4   band2     58245 non-null  float64 
 5   band3     58245 non-null  float64 
 6   band4     58245 non-null  float64 
 7   band5     58245 non-null  float64 
 8   band6     58245 non-null  float64 
 9   band7     58245 non-null  float64 
 10  band8     58245 non-null  float64 
 11  band9     58245 non-null  float64 
 12  band10    58245 non-null  float64 
 13  band11    58245 non-null  float64 
 14  band12    58245 non-null  float64 
 15  band13    58245 non-null  float64 
 16  band14    58245 non-null  float64 
 17  band15    58245 non-null  float64 
 18  band16    58245 non-null  float64 
 19  solarza   58245 non-null  float64 
 20

In [55]:
X_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475367 entries, 0 to 1031459
Data columns (total 24 columns):
 #   Column    Non-Null Count    Dtype   
---  ------    --------------    -----   
 0   hhnn      1475367 non-null  int64   
 1   lon       1475367 non-null  float64 
 2   lat       1475367 non-null  category
 3   band1     1475367 non-null  float64 
 4   band2     1475367 non-null  float64 
 5   band3     1475367 non-null  float64 
 6   band4     1475367 non-null  float64 
 7   band5     1475367 non-null  float64 
 8   band6     1475367 non-null  float64 
 9   band7     1475367 non-null  float64 
 10  band8     1475367 non-null  float64 
 11  band9     1475367 non-null  float64 
 12  band10    1475367 non-null  float64 
 13  band11    1475367 non-null  float64 
 14  band12    1475367 non-null  float64 
 15  band13    1475367 non-null  float64 
 16  band14    1475367 non-null  float64 
 17  band15    1475367 non-null  float64 
 18  band16    1475367 non-null  float64 
 19  

In [8]:
def RMSE(y, y_pred):
    rmse = mean_squared_error(y, y_pred) ** 0.5
    return rmse

In [9]:
import seaborn as sns
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import re
import optuna
from optuna.integration import XGBoostPruningCallback
sns.set_theme(style="darkgrid")

In [45]:
# from optuna.samplers import TPESampler

# sampler = TPESampler(seed=10)

# def objective(trial):
#     dtrain = lgb.Dataset(X_train, label=y_train)
#     dtest = lgb.Dataset(X_test, label=y_test)

#     param = {
#         'objective': 'regression', # 회귀
#         'verbose': -1,
#         'metric': 'rmse', 
#         'max_depth': trial.suggest_int('max_depth',3, 15),
#         'learning_rate': trial.suggest_loguniform("learning_rate", 1e-4, 1e-2),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
#     }

#     model = lgb.LGBMRegressor(**param)
#     lgb_model = model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=25)
#     rmse = RMSE(y_test, lgb_model.predict(X_test))
#     return rmse
        
# study_lgb = optuna.create_study(direction='minimize', sampler=sampler)
# study_lgb.optimize(objective, n_trials=100)

[I 2022-07-28 23:28:26,500] A new study created in memory with name: no-name-e540c7f7-8860-400a-9b5e-e05b998592f7
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-07-28 23:28:39,680] Trial 0 finished with value: 2.282604189992743 and parameters: {'max_depth': 13, 'learning_rate': 0.00011002817226254398, 'n_estim

C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-07-28 23:29:55,500] Trial 8 finished with value: 0.6759579385567136 and parameters: {'max_depth': 10, 'learning_rate': 0.0043508488031796955, 'n_estimators': 478, 'min_child_samples': 87, 'subsample': 0.5520731753100153}. Best is trial 8 with value: 0.675957938556

[I 2022-07-28 23:31:49,488] Trial 16 finished with value: 0.5423403189600985 and parameters: {'max_depth': 13, 'learning_rate': 0.005297127609590053, 'n_estimators': 1981, 'min_child_samples': 43, 'subsample': 0.4037322308791715}. Best is trial 16 with value: 0.5423403189600985.
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of Ligh

C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-07-28 23:34:14,407] Trial 25 finished with value: 0.5266846452030582 and parameters: {'max_depth': 9, 'learning_rate': 0.009657009839995498, 'n_estimators': 1715, 'min_child_samples': 6, 'subsample': 0.47680433020394963}. Best is trial 22 with value: 0.52147160824

[I 2022-07-28 23:36:06,903] Trial 33 finished with value: 0.5506981938706674 and parameters: {'max_depth': 11, 'learning_rate': 0.004625550783139468, 'n_estimators': 1847, 'min_child_samples': 5, 'subsample': 0.4508165834554608}. Best is trial 22 with value: 0.5214716082418841.
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of Light

...

C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-07-28 23:42:35,112] Trial 59 finished with value: 0.5301174585578899 and parameters: {'max_depth': 13, 'learning_rate': 0.0082317666473781, 'n_estimators': 1763, 'min_child_samples': 26, 'subsample': 0.4139493469686168}. Best is trial 22 with value: 0.521471608241

[I 2022-07-28 23:44:32,484] Trial 67 finished with value: 0.5722687515958907 and parameters: {'max_depth': 8, 'learning_rate': 0.0037051083759212858, 'n_estimators': 1413, 'min_child_samples': 22, 'subsample': 0.5194709788219787}. Best is trial 22 with value: 0.5214716082418841.
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of Ligh

C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-07-28 23:47:06,919] Trial 76 finished with value: 0.5395361597090803 and parameters: {'max_depth': 11, 'learning_rate': 0.005956436418024234, 'n_estimators': 1909, 'min_child_samples': 71, 'subsample': 0.4911939093981675}. Best is trial 72 with value: 0.5184938684

[I 2022-07-28 23:49:31,417] Trial 84 finished with value: 0.524210791601338 and parameters: {'max_depth': 13, 'learning_rate': 0.008675887439113546, 'n_estimators': 1954, 'min_child_samples': 68, 'subsample': 0.5804466216476378}. Best is trial 72 with value: 0.5184938684853763.
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of Light

C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-07-28 23:52:07,608] Trial 93 finished with value: 1.2070023851043556 and parameters: {'max_depth': 12, 'learning_rate': 0.000457108525966519, 'n_estimators': 1901, 'min_child_samples': 59, 'subsample': 0.6074360403015646}. Best is trial 72 with value: 0.5184938684

In [46]:
# trial = study_lgb.best_trial
# trial_params = trial.params
# print('Best Trial: score {},\nparams {}'.format(trial.value, trial_params))

Best Trial: score 0.5178388656678076,
params {'max_depth': 14, 'learning_rate': 0.009994761730516164, 'n_estimators': 1997, 'min_child_samples': 73, 'subsample': 0.622413509242086}


In [34]:
categorical_features = ['lat','height','landtype']
feature_names = ['hhnn', 'lon', 'lat', 'band1', 'band2',
       'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9', 'band10',
       'band11', 'band12', 'band13', 'band14', 'band15', 'band16', 'solarza',
       'sateza', 'esr', 'height', 'landtype']

dval = lgb.Dataset(X_test, label=y_test,feature_name=feature_names,
                            categorical_feature=categorical_features)
dtrain = lgb.Dataset(X_train, label=y_train,feature_name=feature_names,
                            categorical_feature=categorical_features)

params = {
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": 511,
    "learning_rate": 0.009994761730516164,
    "colsample_bytree" : 0.9,
    "max_depth" : 14,
    "n_estimators": 1997,             # 400  #1000
    "early_stopping_rounds" : 25,      # 100
    "min_child_samples" :  73,
    "subsample": 0.6224}


# lgbm_reg = LGBMRegressor(objective= "regression",
#                      colsample_bytree = colsample_bytree,
#                      max_depth= max_depth,
#                      n_estimators= n_estimators,
#                      learning_rate= lr,
#                      num_leaves = num_leaves,
#                      min_child_samples = min_child_samples,
#                      subsample = subsample   )

lgbm_reg = lgb.train(params,
    dtrain,
          valid_sets=[dval],
          verbose_eval=20,
                    categorical_feature=categorical_features) 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007409 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4792
[LightGBM] [Info] Number of data points in the train set: 1475367, number of used features: 24
[LightGBM] [Info] Start training from score 1.053687
Training until validation scores don't improve for 25 rounds
[20]	valid_0's rmse: 2.23584
[40]	valid_0's rmse: 1.86133
[60]	valid_0's rmse: 1.55833
[80]	valid_0's rmse: 1.31778
[100]	valid_0's rmse: 1.12839
[120]	valid_0's rmse: 0.980398
[140]	valid_0's rmse: 0.866806
[160]	valid_0's rmse: 0.781691
[180]	valid_0's rmse: 0.718856
[200]	valid_0's rmse: 0.672924
[220]	valid_0's rmse: 0.639399
[240]	valid_0's rmse: 0.615431
[260]	valid_0's rmse: 0.598037
[280]	valid_0's rmse: 0.585511
[300]	valid_0's rmse: 0.57618
[320]	valid_0's rmse: 0.569366
[340]	valid_0's rmse: 0.564009
[360]	valid

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1180]	valid_0's rmse: 0.517592
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1200]	valid_0's rmse: 0.517177
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1220]	valid_0's rmse: 0.516766
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Lig

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1540]	valid_0's rmse: 0.51176
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1820]	valid_0's rmse: 0.508583
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1840]	valid_0's rmse: 0.508385
[LightGBM] [Warning] No further spli

In [35]:
sub = pd.read_csv('./uv_test_cloud.csv')
sub2 = sub[sub['stn'].isin(stn2)]

preds =lgbm_reg.predict(X_test)
mse = mean_squared_error(y_test, preds)
print("MSE: %f" % (mse))

sub2_test = sub2[['hhnn', 'Lon', 'Lat', 'Band1', 'Band2',
       'Band3', 'Band4', 'Band5', 'Band6', 'Band7', 'Band8', 'Band9', 'Band10',
       'Band11', 'Band12', 'Band13', 'Band14', 'Band15', 'Band16', 'SolarZA',
       'SateZA', 'ESR', 'Height', 'LandType']]


# cat_features = ['Lat','Height','LandType']

# for i in enumerate (cat_features) :
#     ca = i[1]
#     sub2_test[ca] = sub2_test[ca].astype('category')

pred2 =lgbm_reg.predict(sub2_test)

MSE: 0.257015


In [37]:
all_inter = pd.DataFrame()
for i in stn1:
    inter = uv_train1[uv_train1['stn'].isin([i])]
    inter = inter.interpolate(method ='linear') 
    all_inter = pd.concat([all_inter, inter])
    
    
uv_train1 = all_inter.sort_index()

uv_train1_val = uv_train1[(uv_train1['yyyymmdd'] >= 20210501) & (uv_train1['yyyymmdd'] < 20210801)]
idx1 = uv_train1_val.index
all_group1 = uv_train1.drop(uv_train1.index[idx1])



X = all_group1[['hhnn', 'lon', 'lat', 'band1', 'band2',
       'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9', 'band10',
       'band11', 'band12', 'band13', 'band14', 'band15', 'band16', 'solarza',
       'sateza', 'esr', 'height', 'landtype', 'total_cloud', 'ml_cloud']]
Y = all_group1['uv']

x = uv_train1_val[['hhnn', 'lon', 'lat', 'band1', 'band2',
       'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9', 'band10',
       'band11', 'band12', 'band13', 'band14', 'band15', 'band16', 'solarza',
       'sateza', 'esr', 'height', 'landtype', 'total_cloud', 'ml_cloud']]
y = uv_train1_val['uv']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123)

X_train = pd.concat([X,X_train])
y_train = pd.concat([Y,y_train])




In [38]:
categorical_features = ['lat','height','landtype']
feature_names = ['hhnn', 'lon', 'lat', 'band1', 'band2',
       'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9', 'band10',
       'band11', 'band12', 'band13', 'band14', 'band15', 'band16', 'solarza',
       'sateza', 'esr', 'height', 'landtype', 'total_cloud', 'ml_cloud']

dval = lgb.Dataset(X_test, label=y_test,feature_name=feature_names,
                            categorical_feature=categorical_features)
dtrain = lgb.Dataset(X_train, label=y_train,feature_name=feature_names,
                            categorical_feature=categorical_features)

params = {
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": 511,
    "learning_rate": 0.009994761730516164,
    "colsample_bytree" : 0.9,
    "max_depth" : 14,
    "n_estimators": 1997,             # 400  #1000
    "early_stopping_rounds" : 25,      # 100
    "min_child_samples" :  73,
    "subsample": 0.6224}


# lgbm_reg = LGBMRegressor(objective= "regression",
#                      colsample_bytree = colsample_bytree,
#                      max_depth= max_depth,
#                      n_estimators= n_estimators,
#                      learning_rate= lr,
#                      num_leaves = num_leaves,
#                      min_child_samples = min_child_samples,
#                      subsample = subsample   )

lgbm_reg = lgb.train(params,
    dtrain,
          valid_sets=[dval],
          verbose_eval=20,
                    categorical_feature=categorical_features) 

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030720 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5171
[LightGBM] [Info] Number of data points in the train set: 1279598, number of used features: 26
[LightGBM] [Info] Start training from score 1.103047
Training until validation scores don't improve for 25 rounds
[20]	valid_0's rmse: 2.22288
[40]	valid_0's rmse: 1.84632
[60]	valid_0's rmse: 1.5419
[80]	valid_0's rmse: 1.29929
[100]	valid_0's rmse: 1.10882
[120]	valid_0's rmse: 0.957015
[140]	valid_0's rmse: 0.840832
[160]	valid_0's rmse: 0.750514
[180]	valid_0's rmse: 0.682203
[200]	valid_0's rmse: 0.63099
[220]	valid_0's rmse: 0.593192
[240]	valid_0's rmse: 0.564712
[260]	valid_0's rmse: 0.543643
[280]	valid_0's rmse: 0.527879
[300]	valid_0's rmse: 0.515765
[320]	valid_0's rmse: 0.506299
[340]	valid_0's rmse: 0.498778
[360]	valid_0's rmse: 0.492514
[380]	valid_0's rmse: 0.487319
[400]	valid_0'

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1260]	valid_0's rmse: 0.408445
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1280]	valid_0's rmse: 0.407564
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1580]	valid_0's rmse: 0.39609
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[1800]	valid_0's rmse: 0.389246
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1820]	valid_0's rmse: 0.388641
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Did not meet early stopping. Best iteration is:
[1997]	valid_0's rmse: 0.384137


In [39]:
sub = pd.read_csv('./uv_test_cloud.csv')
sub.columns

Index(['yyyymmdd', 'hhnn', 'stn', 'Lon', 'Lat', 'UV', 'Band1', 'Band2',
       'Band3', 'Band4', 'Band5', 'Band6', 'Band7', 'Band8', 'Band9', 'Band10',
       'Band11', 'Band12', 'Band13', 'Band14', 'Band15', 'Band16', 'SolarZA',
       'SateZA', 'ESR', 'Height', 'LandType', 'total_cloud', 'ml_cloud',
       'rainfall'],
      dtype='object')

In [40]:
sub = pd.read_csv('./uv_test_cloud.csv')
sub = sub.drop(['rainfall'], axis=1)

all_inter = pd.DataFrame()
for i in stn1:
    inter = sub[sub['stn'].isin([i])]
    inter = inter.interpolate(method ='linear') 
    all_inter = pd.concat([all_inter, inter])
    
    
sub1 = all_inter[all_inter['stn'].isin(stn1)]

preds =lgbm_reg.predict(X_test)
mse = mean_squared_error(y_test, preds)
print("MSE: %f" % (mse))

sub1_test = sub1[['hhnn', 'Lon', 'Lat', 'Band1', 'Band2',
       'Band3', 'Band4', 'Band5', 'Band6', 'Band7', 'Band8', 'Band9', 'Band10',
       'Band11', 'Band12', 'Band13', 'Band14', 'Band15', 'Band16', 'SolarZA',
       'SateZA', 'ESR', 'Height', 'LandType', 'total_cloud', 'ml_cloud',
       ]]


pred1 =lgbm_reg.predict(sub1_test)

MSE: 0.147561


In [41]:
sub1['UV'] = pred1
sub2['UV'] = pred2
pred_end = pd.concat([sub1,sub2])
pred_=pred_end.sort_index(ascending=True)

submission = pd.read_csv('./1-1_검증데이터셋.csv')
submission['UV']=pred_[['UV']]
#pred_[['UV']].hist()
submission.to_csv('220224_cate1.csv', index=False) #0.599471

In [37]:
abs_sub = submission.abs()
abs_sub

,YearMonthDayHourMinute,STN,UV
0,202206010000,13,0.001590
1,202206010000,105,0.000162
2,202206010000,108,0.000835
3,202206010000,112,0.002119
4,202206010000,115,0.001370
...,...,...,...
51850,202206250000,146,0.026009
51851,202206250000,152,0.000971
51852,202206250000,156,0.004248
51853,202206250000,159,0.003738


In [38]:
abs_sub.to_csv('220224.csv', index=False) #0.600531

In [39]:

pred_hhnn = pred_.copy()
condition1 = (pred_hhnn.hhnn <=550) & (pred_hhnn.hhnn >= 0)
pred_hhnn.loc[condition1, 'UV']=0

condition2 = (pred_hhnn.hhnn <=2350) & (pred_hhnn.hhnn >= 2000)
pred_hhnn.loc[condition2, 'UV']=0

submission = pd.read_csv('./1-1_검증데이터셋.csv')
submission['UV']=pred_hhnn[['UV']]
 
submission.to_csv('220224.csv', index=False) #0.600548